In [28]:
%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import requests
from IPython.display import Image





In [30]:
def get_pokemon_info(name):

    base_url = "https://pokeapi.co/api/v2/"

    url = f"{base_url}/pokemon/{name}"
    response = requests.get(url)

    if response.status_code == 200:

        pokemon_info =  response.json()
        url1 = pokemon_info["sprites"]['front_default']
        url2 = pokemon_info['types'][0]['type']['name']
        
        return [url1, url2]

    else:
        print(f"Failed to retrieve data. Error {response.status_code}")


In [31]:
# from certificate import Certificate
# from wallet import Wallet

# Alice = Wallet()
# cert = Certificate(Alice.publicKey)
# Alice.sign(cert)
# cert.is_legit()


# INITIALISATION

In [32]:
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake

walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

walletAlice = Wallet()
walletSacha = Wallet()
walletTiméo = Wallet()

pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)




# Smart contract importation

In [33]:
from smart_contract import SmartContractDefinition, SmartContractWritingOperation
from CollectionNFT import Collection
from certificate import Certificate
import inspect


reset_blockchain()
collection = inspect.getsource(Collection)
dummySmartContractDefinition = SmartContractDefinition(walletSacha.publicKey, collection)
walletAlice.sign(dummySmartContractDefinition)

collection_nft = dummySmartContractDefinition.instantiate_collection(8)




    
# change_ownership_SmartContractWritingOperation = SmartContractWritingOperation(
#     walletAlice.publicKey,
#     collection.hash(),
#     'change_ownership',
#     ['receiver publicKey']
# )


# open_mint_SmartContractWritingOperation = SmartContractWritingOperation(
#     walletAlice.publicKey,
#     collection.hash(),
#     'open_minting',
#     ['time in seconds']
# )






{'self': <smart_contract.SmartContractDefinition object at 0x0000026F1B533440>, 'size': 8, 'Collection': <class 'smart_contract.Collection'>}


Users can “mint” tokens under certain conditions :  
- Once the limit of N has been reached, there is no minting allowed anymore. 
- There should be a “mint” period. The owner of the collection can open the collection

In [34]:
open_minting_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'open_minting',
    [10000]
)
walletAlice.sign(open_minting_SmartContractWritingOperation)
open_minting_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(open_minting_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)



    

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'mint_token',
    []
)
walletAlice.sign(mint_token_SmartContractWritingOperation)
mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(mint_token_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

[10000]
Minting is open
[]
Minting
Minted !
ownership changed


In [35]:
mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'mint_token',
    []
)
walletAlice.sign(mint_token_SmartContractWritingOperation)
mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(mint_token_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

[]
Minting
Minted !
ownership changed


In [36]:
open_minting_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'open_minting',
    [10000]
)
walletAlice.sign(open_minting_SmartContractWritingOperation)
open_minting_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(open_minting_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)



    

mint_token_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'mint_token',
    []
)

for i in range (10):
    walletAlice.sign(mint_token_SmartContractWritingOperation)
    mint_token_SmartContractWritingOperation.apply_on_contract(collection_nft)
    node1.new_certificate(mint_token_SmartContractWritingOperation)


    for i in range(4):
        certificate = Certificate(walletAlice.publicKey)
        walletAlice.sign(certificate)
        node1.new_certificate(certificate)


[10000]
Minting is open
[]
Minting
Minted !
ownership changed
[]
Minting
Minted !
ownership changed
[]
Minting
Minted !
ownership changed
[]
Minting
Minted !
ownership changed
[]
Minting
Minted !
ownership changed
[]
Minting
Minted !
ownership changed
[]
Limit of tokens reached
ownership changed
[]
Limit of tokens reached
ownership changed
[]
Limit of tokens reached
ownership changed
[]
Limit of tokens reached
ownership changed


Create a collection of N tokens that all have a distinguishable identifier

In [41]:
print(collection_nft.tokens)
for token in collection_nft.tokens:
    print(token)

{'Token No0': {'name': 'pikachu', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png', 'description': 'electric'}, 'Token No1': {'name': 'bulbasaur', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png', 'description': 'grass'}, 'Token No2': {'name': 'charmander', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/4.png', 'description': 'fire'}, 'Token No3': {'name': 'squirtle', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/7.png', 'description': 'water'}, 'Token No4': {'name': 'caterpie', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/10.png', 'description': 'bug'}, 'Token No5': {'name': 'pidgey', 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/16.png', 'description': 'normal'}, 'Token No6': {'name': 'rattata', 'image_link': 'https://raw.githu

Each token must be linked with a unique image or a description of a game item

In [42]:
collection_nft.tokens['Token No3']

{'name': 'squirtle',
 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/7.png',
 'description': 'water'}

In [44]:
Image(url=collection_nft.tokens['Token No3']['image_link'])

We need to be able to know at any time who is owning which of those tokens

In [38]:
change_ownership_SmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'change_ownership',
    ['hash token',walletTiméo.publicKey]
)
walletAlice.sign(change_ownership_SmartContractWritingOperation)
change_ownership_SmartContractWritingOperation.apply_on_contract(collection_nft)
node1.new_certificate(change_ownership_SmartContractWritingOperation)


for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

['hash token', '30820122300d06092a864886f70d01010105000382010f003082010a0282010100b8662070edcf22d3d7a345d20ae30ee66491ff7f74690b3f9b554ea20a71467f8ef9fcfbd5150d2ef478808fd27c68c79953db9de40109b0c07b9dcf24b04f8db9e66ce6c4bbc0cbff3327da6c27e0859343039377692eafb417e6499165710d5c24fa7092b63f2a63b159e03f6afee946d234e5767549d3e7214e895151d0192abdb09087d350fc10c57b542c5c94e419dc3164e42cd5246423642b3d4fed909f330b7db96a2ed12388e39fc2fb643e0bdbc7daabb7efd664fb42880fb35e00989588de76d41f706e7e0944ecff41fd9180cae1e4f57b1209f45e86bd0288376237a48a25b2a9da2318cf9da0aafd231096bc3e2edcf30eb2bffd4a9ca74e710203010001']
ownership changed


Each token must be linked with a unique image or a description of a game item

In [39]:


smartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(
    node1.blockchain,
    dummySmartContractDefinition.hash()
)


assert smartContractObject.messages[0][11:] == "Bonjour!"

"Success!"

{'type': 'Block', 'issuer': '...0000000000000000...', 'timestamp': 0, 'hash': '998cc8c77c2c1f9b...', 'signature': '...', 'index': 0, 'parent': '0000000000000000...', 'certificates': [], 'nonce': 0}
{'type': 'Block', 'issuer': '...7f0491fe788fdcfb...', 'timestamp': 1741205971638, 'hash': '8f294a504c13081d...', 'signature': '0fb3a49af65c38bf...', 'index': 1, 'parent': '998cc8c77c2c1f9b...', 'certificates': [{'type': 'Certificate', 'issuer': '...c1b64d20ace1dbf0...', 'timestamp': 1741205971477, 'hash': '7ab6ad9fe92702da...', 'signature': '3ba9d6112a09af41...'}, {'type': 'SmartContractWritingOperation', 'issuer': '...c1b64d20ace1dbf0...', 'timestamp': 1741205971435, 'hash': 'eb66889adc48bb50...', 'signature': 'cd4dac4b3f954b66...'}, {'type': 'Certificate', 'issuer': '...c1b64d20ace1dbf0...', 'timestamp': 1741205971600, 'hash': '364ffae4bd96b04d...', 'signature': '0cdb41122c04089c...'}, {'type': 'Certificate', 'issuer': '...c1b64d20ace1dbf0...', 'timestamp': 1741205971518, 'hash': 'fb985764

AttributeError: 'NoneType' object has no attribute 'messages'

In [ ]:
from certificate import Certificate



for i in range(4):
    certificate = Certificate(walletAlice.publicKey)
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)
    

In [ ]:
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("pikachu")[0], get_pokemon_info("pikachu")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("bulbasaur")[0], get_pokemon_info("bulbasaur")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("charmander")[0], get_pokemon_info("charmander")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("squirtle")[0], get_pokemon_info("squirtle")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("caterpie")[0], get_pokemon_info("caterpie")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("pidgey")[0], get_pokemon_info("pidgey")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("rattata")[0], get_pokemon_info("rattata")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("spearow")[0], get_pokemon_info("spearow")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("ekans")[0], get_pokemon_info("ekans")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("sandshrew")[0], get_pokemon_info("sandshrew")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("clefairy")[0], get_pokemon_info("clefairy")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("vulpix")[0], get_pokemon_info("vulpix")[1])


Minting
Minted !
Minting
Minted !
Minting
Minted !
Minting
Minted !
Minting
Minted !
Minting
Minted !
Minting
Minted !
Minting
Minted !
Minting
Minted !
Minting
Minted !
Limit of tokens reached
Limit of tokens reached


In [ ]:
CollectionA.display()

{'Token No0': {'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png',
  'description': 'electric',
  'can_be_exchanged': False,
  'identifier': '59118dc5174852c28d73cfaaf7b5f3ebc957ca35d06854d5212cf01a06d91fd9',
  'owner': 'George'},
 'Token No1': {'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png',
  'description': 'grass',
  'can_be_exchanged': True,
  'identifier': 'd8fed864393e81a96f7c6587db5933f6f745715b9037550e21475ce1f32b5887',
  'owner': 'George'},
 'Token No2': {'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/4.png',
  'description': 'fire',
  'can_be_exchanged': False,
  'identifier': 'e5e4e3f2623e4a45bd7f9b0da0630320672ad13e8efad08ea70c732a3092f63c',
  'owner': 'George'},
 'Token No3': {'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/7.png',
  'description': 'water',
  'can_be_exchanged': True,
  'identifier': '

In [ ]:
CollectionA.change_ownership("Mike","289819341682e319d27e8e5f0c57e4f2ab315617b4c5ea97012df6a6ddb4e1b9")

In [ ]:
CollectionA.display()['Token No3']

{'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/7.png',
 'description': 'water',
 'can_be_exchanged': True,
 'identifier': '289819341682e319d27e8e5f0c57e4f2ab315617b4c5ea97012df6a6ddb4e1b9',
 'owner': 'Mike'}

In [ ]:
Image(url=CollectionA.display()['Token No4']['image_link'])

In [ ]:
CollectionA.open_minting(5)

Minting is open


In [ ]:
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("pikachu")[0], get_pokemon_info("pikachu")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("bulbasaur")[0], get_pokemon_info("bulbasaur")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("charmander")[0], get_pokemon_info("charmander")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("squirtle")[0], get_pokemon_info("squirtle")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("caterpie")[0], get_pokemon_info("caterpie")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("pidgey")[0], get_pokemon_info("pidgey")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("rattata")[0], get_pokemon_info("rattata")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("spearow")[0], get_pokemon_info("spearow")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("ekans")[0], get_pokemon_info("ekans")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("sandshrew")[0], get_pokemon_info("sandshrew")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("clefairy")[0], get_pokemon_info("clefairy")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("vulpix")[0], get_pokemon_info("vulpix")[1])



Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol
Minting is closed lol


In [ ]:
CollectionA = Collection("George", 10)
CollectionA.open_minting(1)
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("pikachu")[0], get_pokemon_info("pikachu")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("bulbasaur")[0], get_pokemon_info("bulbasaur")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("charmander")[0], get_pokemon_info("charmander")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("squirtle")[0], get_pokemon_info("squirtle")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("caterpie")[0], get_pokemon_info("caterpie")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("pidgey")[0], get_pokemon_info("pidgey")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("rattata")[0], get_pokemon_info("rattata")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("spearow")[0], get_pokemon_info("spearow")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("ekans")[0], get_pokemon_info("ekans")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("sandshrew")[0], get_pokemon_info("sandshrew")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("clefairy")[0], get_pokemon_info("clefairy")[1])
CollectionA.mint_token(CollectionA.CollectionOwner, get_pokemon_info("vulpix")[0], get_pokemon_info("vulpix")[1])


Minting is open
Minting
[{'identifier': 0, 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png', 'description': 'electric', 'owner': 'George'}]
Minted !
Minting
[{'identifier': 0, 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png', 'description': 'electric', 'owner': 'George'}, {'identifier': 1, 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png', 'description': 'grass', 'owner': 'George'}]
Minted !
Minting
[{'identifier': 0, 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png', 'description': 'electric', 'owner': 'George'}, {'identifier': 1, 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png', 'description': 'grass', 'owner': 'George'}, {'identifier': 2, 'image_link': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/4.png', 'description': 'fire', 